# Load modules

In [ ]:
import sys, os
import gc
import time
import random
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# scikit-learn
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error as mae

# scipy
# from scipy.optimize import minimize, BFGS, LinearConstraint
from scipy.optimize import minimize, LinearConstraint

In [ ]:
# original modules
from annin_dofu import calc, matrix, parallel, stats, utils

# approximation系クラスのimport
from annin_dofu.response_surface_methodology import Approximation2dFunction as a2f
from annin_dofu.response_surface_methodology import ApproximationRBFInterpolation as ari

# Configuration

## random seed

In [ ]:
np.random.seed(57)
random.seed(57)

## warningの非表示

In [ ]:
# warningの非表示
import warnings
warnings.filterwarnings('ignore')

## 左寄せにするマジックコマンド

In [ ]:
%%html
<style>
    table{float:left}
    .MathJax{float: left;}
</style>

## データフレームの表示設定

In [ ]:
# データフレームの表示行数, 表示列数設定.
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

# カラム内の文字数設定. デフォルトは50.
pd.set_option('display.max_colwidth', 100)

## カレントディレクトリの設定

In [ ]:
# 念の為カレントディレクトリをファイルの場所に変更しておく
os.chdir(os.getcwd())

## 経過時間取得のための開始時間保存

In [ ]:
start_time = time.time()

# Constants

## 円周率

In [ ]:
# 円周率の定義
pi = np.pi

# 1-dim sample

- 1次元データのサンプル

## make sample data

### サンプルサイズ等の定義

In [ ]:
train_n_sample_grid1d = 200
train_n_sample_grid1dlr = 50
gaussian_noize_std1d = 0.1

### データ作成

In [ ]:
def d1_func0(x):
    return np.sin(2*x/pi)

In [ ]:
x_1d = np.linspace(-10, 10, num=train_n_sample_grid1d)
x_1d_ = x_1d.reshape([1, train_n_sample_grid1d]).T
sin_y_1d = d1_func0(x_1d)
sin_y_1do = sin_y_1d + 2 * gaussian_noize_std1d
sin_y_1du = sin_y_1d - 2 * gaussian_noize_std1d
y_1d = np.random.normal(sin_y_1d, gaussian_noize_std1d, size=sin_y_1d.shape)

In [ ]:
x_1dl = np.linspace(-15, -10, num=train_n_sample_grid1dlr)[:-1]
x_1dl_ = x_1dl.reshape([1, len(x_1dl)]).T
sin_y_1dl = d1_func0(x_1dl)
sin_y_1dlo = sin_y_1dl + 2 * gaussian_noize_std1d
sin_y_1dlu = sin_y_1dl - 2 * gaussian_noize_std1d
y_1dl = np.random.normal(sin_y_1dl, gaussian_noize_std1d, size=sin_y_1dl.shape)

In [ ]:
x_1dr = np.linspace(10, 15, num=train_n_sample_grid1dlr)[1:]
x_1dr_ = x_1dr.reshape([1, len(x_1dr)]).T
sin_y_1dr = d1_func0(x_1dr)
sin_y_1dro = sin_y_1dr + 2 * gaussian_noize_std1d
sin_y_1dru = sin_y_1dr - 2 * gaussian_noize_std1d
y_1dr = np.random.normal(sin_y_1dr, gaussian_noize_std1d, size=sin_y_1dr.shape)

In [ ]:
all_x_1d = np.hstack([x_1dl, x_1d, x_1dr])
all_x_1d_ = all_x_1d.reshape([1, train_n_sample_grid1d+2*train_n_sample_grid1dlr-2]).T
all_y_1d = np.hstack([y_1dl, y_1d, y_1dr])
all_sin_y_1d = d1_func0(all_x_1d)
all_sin_y_1do = all_sin_y_1d + 2 * gaussian_noize_std1d
all_sin_y_1du = all_sin_y_1d - 2 * gaussian_noize_std1d

all_x_1d.shape, all_x_1d_.shape, all_y_1d.shape

# 1-dim sample modeling

## 2D function approximation

In [ ]:
a2f_reg1d = a2f()
a2f_reg1d.fit(x_1d_, y_1d)
a2f_pred1d = a2f_reg1d.predict(all_x_1d_)

In [ ]:
plt.figure(figsize=(16, 9))

# 実績
plt.fill_between(all_x_1d, all_sin_y_1du, all_sin_y_1do, color='orange', lw=2, label='sine 2sigma-neighborhood', alpha=0.5)
plt.scatter(all_x_1d, all_y_1d, color='red', lw=2, label='sine + gaussian noize')

# 予測
plt.plot(all_x_1d, a2f_pred1d, color='blue', lw=4, label='2D approximation')

plt.ylim([-1.75, 1.75])

plt.legend(bbox_to_anchor=(0, 1), loc='upper left', borderaxespad=0.5, fontsize=15)

plt.grid()
plt.title('d2func_approximation_1Dsample', fontsize=20)

## Gaussian RBF interpolate approximation

In [ ]:
n_samples_list = [20, 50]
beta_val_list = [0, 0.1, 1, 10]
lambda_val_list = [0, 0.1, 1, 10]

In [ ]:
n_ii = len(n_samples_list)
n_jj = len(beta_val_list)
n_kk = len(lambda_val_list)


for ii in range(n_ii):
    
    plt.figure(
        figsize=(n_jj*16, n_kk*9),
        facecolor='white',
        dpi=150
    )

    # 余白を設定
    plt.subplots_adjust(wspace=0.1, hspace=0.15, left=0.05, right=0.95, bottom=0.05, top=0.95)
    
    num_hyper = 0
    num_plt = 1
    n_samples = n_samples_list[ii]
    
    # hyper parameters
    for jj in range(n_jj):
        for kk in range(n_kk):
            
            beta_val = beta_val_list[jj]
            lambda_val = lambda_val_list[kk]
            
            if num_hyper==0:
                ari_reg1d = ari(
                    rbf='gaussian',
                    beta_val=beta_val,
                    lambda_val=lambda_val,
                    n_samples=n_samples,
                    rel_ct='exclusive'
                )
            else:
                ari_reg1d.set_params(**{
                    'beta_val': beta_val,
                    'lambda_val': lambda_val
                })
            
            ari_reg1d.fit(x_1d_, y_1d)
            ari_pred1d = ari_reg1d.predict(all_x_1d_)
            ari_pred1d_centers_x = ari_reg1d.center_x_arr
            ari_pred1d_centers_y = ari_reg1d.center_y_arr
            
            # 可視化
            # 行数, 列数
            plt.subplot(n_kk, n_jj, num_plt)

            # 実績
            plt.fill_between(all_x_1d, all_sin_y_1du, all_sin_y_1do, color='orange', lw=2, label='sine 2sigma-neighborhood', alpha=0.5)
            plt.plot(all_x_1d, all_y_1d, color='red', lw=2, label='sine + Gaussian noize')

            # 予測
            plt.plot(all_x_1d, ari_pred1d, color='blue', lw=4, label='Gaussian RBF approximation')
            plt.scatter(ari_pred1d_centers_x, ari_pred1d_centers_y, color='blue', lw=5, label='center points of RBF')
            
            plt.ylim([-1.75, 1.75])

            plt.legend(bbox_to_anchor=(0, 1), loc='upper left', borderaxespad=0.5, fontsize=15)
            plt.grid()
            
            #title = 'n_samples:'+str(n_samples)+', beta_val:'+str(beta_val)+', lambda_val:'+str(lambda_val)
            title = 'Gaussian RBF (N, beta, lambda) = ('+str(n_samples)+', '+str(beta_val)+', '+str(lambda_val)+')'
            plt.title(label=title, fontdict={'fontsize': 20})
            
            num_plt += 1
            num_hyper += 1

## Thin Plate Spline RBF interpolate approximation

In [ ]:
n_samples_list = [20, 50]
beta_val_list = [0]
lambda_val_list = [0, 0.1, 1, 10]

In [ ]:
n_ii = len(n_samples_list)
n_jj = len(beta_val_list)
n_kk = len(lambda_val_list)


for ii in range(n_ii):
    
    plt.figure(
        figsize=(n_jj*16, n_kk*9),
        facecolor='white',
        dpi=150
    )

    # 余白を設定
    plt.subplots_adjust(wspace=0.1, hspace=0.15, left=0.05, right=0.95, bottom=0.05, top=0.95)
    
    num_hyper = 0
    num_plt = 1
    n_samples = n_samples_list[ii]
    
    # hyper parameters
    for jj in range(n_jj):
        for kk in range(n_kk):
            
            beta_val = beta_val_list[jj]
            lambda_val = lambda_val_list[kk]
            
            if num_hyper==0:
                ari_reg1d = ari(
                    rbf='TPS',
                    beta_val=beta_val,
                    lambda_val=lambda_val,
                    n_samples=n_samples,
                    rel_ct='exclusive'
                )
            else:
                ari_reg1d.set_params(**{
                    'beta_val': beta_val,
                    'lambda_val': lambda_val
                })
            
            ari_reg1d.fit(x_1d_, y_1d)
            ari_pred1d = ari_reg1d.predict(all_x_1d_)
            ari_pred1d_centers_x = ari_reg1d.center_x_arr
            ari_pred1d_centers_y = ari_reg1d.center_y_arr
            
            # 可視化
            # 行数, 列数
            plt.subplot(n_kk, n_jj, num_plt)

            # 実績
            plt.fill_between(all_x_1d, all_sin_y_1du, all_sin_y_1do, color='orange', lw=2, label='sine 2sigma-neighborhood', alpha=0.5)
            plt.plot(all_x_1d, all_y_1d, color='red', lw=2, label='sine + TPS noize')

            # 予測
            plt.plot(all_x_1d, ari_pred1d, color='blue', lw=4, label='TPS RBF approximation')
            plt.scatter(ari_pred1d_centers_x, ari_pred1d_centers_y, color='blue', lw=5, label='center points of RBF')
            
            plt.ylim([-1.75, 1.75])

            plt.legend(bbox_to_anchor=(0, 1), loc='upper left', borderaxespad=0.5, fontsize=15)
            plt.grid()
            
            #title = 'n_samples:'+str(n_samples)+', beta_val:'+str(beta_val)+', lambda_val:'+str(lambda_val)
            title = 'Thin Plate Spline RBF (N, lambda) = ('+str(n_samples)+', '+str(lambda_val)+')'
            plt.title(label=title, fontdict={'fontsize': 20})
            
            num_plt += 1
            num_hyper += 1

# 2-dim sample

- 2次元データのサンプル

## make sample data

### train作成

In [ ]:
n_dim = 2
train_n_sample_grid = 20
train_n_sample = train_n_sample_grid ** n_dim
gaussian_noize_std = 0.1
#gaussian_noize_std = 0

# (x,y)データを作成
train_x1 = np.linspace(-3*pi, 3*pi, train_n_sample_grid)
train_x2 = np.linspace(-3*pi, 3*pi, train_n_sample_grid)

# 格子点を作成
train_X1, train_X2 = np.meshgrid(train_x1, train_x2)

# sampleの計算式
train_Y = np.cos(train_X1/pi) * np.sin(train_X2/pi)
train_Y += np.random.normal(0, gaussian_noize_std, size=train_Y.shape)

print('train sample size:', train_n_sample)

### test作成

In [ ]:
test_n_sample_grid = 100
test_n_sample = test_n_sample_grid ** n_dim

# (x,y)データを作成
test_x1 = np.linspace(-3*pi, 3*pi, test_n_sample_grid)
test_x2 = np.linspace(-3*pi, 3*pi, test_n_sample_grid)

# 格子点を作成
test_X1, test_X2 = np.meshgrid(test_x1, test_x2)

# sampleの計算式
test_Y = np.cos(test_X1/pi) * np.sin(test_X2/pi)
test_Y += np.random.normal(0, gaussian_noize_std, size=test_Y.shape)

print('test sample size:', test_n_sample)

### wide版(long)作成

In [ ]:
long_n_sample_grid = 150
long_n_sample = long_n_sample_grid ** n_dim

# (x,y)データを作成
long_x1 = np.linspace(-30, 30, long_n_sample_grid)
long_x2 = np.linspace(-30, 30, long_n_sample_grid)

# 格子点を作成
long_X1, long_X2 = np.meshgrid(long_x1, long_x2)

# sampleの計算式
long_Y = np.cos(long_X1/pi) * np.sin(long_X2/pi)
long_Y += np.random.normal(0, gaussian_noize_std, size=long_Y.shape)

print('long sample size:', long_n_sample)

## ndarray変換

In [ ]:
train_x_arr = np.array([train_X1.reshape(train_n_sample), train_X2.reshape(train_n_sample)]).T
train_y_arr = train_Y.reshape(train_n_sample_grid**n_dim)

In [ ]:
test_x_arr = np.array([test_X1.reshape(test_n_sample), test_X2.reshape(test_n_sample)]).T
test_y_arr = test_Y.reshape(test_n_sample_grid**n_dim)

In [ ]:
long_x_arr = np.array([long_X1.reshape(long_n_sample), long_X2.reshape(long_n_sample)]).T
long_y_arr = long_Y.reshape(long_n_sample_grid**n_dim)

## 可視化

### train

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize = (12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('2D sample train data', size=20)

# 軸ラベルを設定
ax.set_xlabel('train_x1', size = 16)
ax.set_ylabel('train_x2', size = 16)
ax.set_zlabel('train_y', size = 16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, train_Y, colors = 'black', offset = -1)

plt.show()

### test

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize = (12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('2D sample test data', size=20)

# 軸ラベルを設定
ax.set_xlabel('test_x1', size = 16)
ax.set_ylabel('test_x2', size = 16)
ax.set_zlabel('test_y', size = 16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, test_Y, colors = 'black', offset = -1)

plt.show()

### train + test

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize = (12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('2D sample train and test data', size=20)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('y', size = 16)

# 曲面を描画
#ax.plot_surface(train_X1, train_X2, train_Y, cmap='jet', alpha=0.5)
ax.plot_surface(train_X1, train_X2, train_Y, alpha=0.5, edgecolor='black')
ax.plot_surface(test_X1, test_X2, test_Y, cmap='ocean', alpha=0.5)

# 底面に等高線を描画
ax.contour(test_X1, test_X2, test_Y, colors = 'black', offset = -1)

plt.show()

### wide版

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize = (12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('2D sample wide data', size=20)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('wide_y', size = 16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, long_Y, colors = 'black', offset = -1)

plt.show()

# 2-dim sample modeling

## 2D function approximation

In [ ]:
a2f_reg = a2f()
a2f_reg.fit(train_x_arr, train_y_arr)

In [ ]:
a2fpred_train_y_arr = a2f_reg.predict(train_x_arr)
a2fpred_test_y_arr = a2f_reg.predict(test_x_arr)

In [ ]:
pd.DataFrame(a2f_reg.opt_A_mat)

## Gaussian RBF interpolate approximation

### grid search

In [ ]:
gaussian_gs_start = time.time()

In [ ]:
gs_kf = KFold(n_splits=2, shuffle=True, random_state=57)

In [ ]:
gs_reg0 = ari(
    rbf='gaussian',
    gtol=1e-8,
    n_samples=100,
    rel_ct='exclusive',
    sampling_method='random',
    random_state=57,
    print_debug=False
)

In [ ]:
gaussian_ari_param_grid0 = {
    #'w0_arr': w0_patterns,
    'beta_val': [0.01, 0.1, 0.5, 1],
    'lambda_val': [0, 0.1, 0.5, 1]
    #'n_samples': [10, 20, 30],
    #'sampling_method': ['random', 'last']
}

In [ ]:
# grid searchの実行
gs0 = GridSearchCV(
    gs_reg0,
    param_grid=gaussian_ari_param_grid0,
    scoring='neg_mean_absolute_error',
    # cv=2
    cv=gs_kf
)
gs0.fit(train_x_arr, train_y_arr)

In [ ]:
# 最適なパラメータとスコアの確認
gs0.best_params_, gs0.best_score_, gs0.best_estimator_.params

In [ ]:
gs0.cv_results_

In [ ]:
best_gaussian_ari_params0 = gs0.best_estimator_.get_params()
print('best params:')
print(best_gaussian_ari_params0)

In [ ]:
gs_reg1 = ari(**best_gaussian_ari_params0)
gs_reg1.fit(train_x_arr, train_y_arr)
gs_reg1.get_params()

In [ ]:
# 重み初期値候補
w0_patterns = list(stats.temp_multivar_normal_dist_sampling(gs_reg1.params['n_samples'], 3, mean_val=1, var_val=4))
w0_patterns += [np.zeros(gs_reg1.params['n_samples'])]
print(w0_patterns[:5])

In [ ]:
gaussian_ari_param_grid1 = {
    'w0_arr': w0_patterns
}

In [ ]:
# 重み初期値についてgrid searchを実行する.
gs1 = GridSearchCV(
    gs_reg1,
    param_grid=gaussian_ari_param_grid1,
    scoring='neg_mean_absolute_error',
    # cv=2
    cv=gs_kf
)
gs1.fit(train_x_arr, train_y_arr)

In [ ]:
# 最適なパラメータとスコアの確認
gs1.best_params_, gs1.best_score_

In [ ]:
best_gaussian_ari_params1 = gs1.best_estimator_.get_params()
print('best params:')
print(best_gaussian_ari_params1)

In [ ]:
utils.get_elapsed_time(gaussian_gs_start)

### modeling

In [ ]:
gaussian_ari_params0 = best_gaussian_ari_params1

In [ ]:
# best paramsを設定してモデリング
gaussian_ari_reg = ari(**gaussian_ari_params0)
gaussian_ari_reg.fit(train_x_arr, train_y_arr)

In [ ]:
gaussian_ari_reg.get_params()

In [ ]:
gaussian_aripred_train_y_arr = gaussian_ari_reg.predict(train_x_arr)
gaussian_aripred_test_y_arr = gaussian_ari_reg.predict(test_x_arr)

## Thin Plate Spline RBF interpolate approximation

In [ ]:
trbf = ari().thin_plate_spline_rbf
trbf

In [ ]:
x_ = np.arange(-1, 1.01, 0.01)
#y_ = [trbf(np.array([0]), np.array([i])) for i in x_]
y_ = [trbf(0, i, beta_val=10) for i in x_]
plt.plot(x_, y_)

### grid search

In [ ]:
tps_gs_start = time.time()

In [ ]:
gs_kf = KFold(n_splits=2, shuffle=True, random_state=57)

In [ ]:
#gs_reg0 = ari(rbf='TPS', gtol=1e-8, n_samples=100, rel_ct='equal', sampling_method='random', random_state=57, print_debug=False)
#gs_reg0 = ari(rbf='TPS', gtol=1e-8, n_samples=100, rel_ct='all', sampling_method='random', random_state=57, print_debug=False)
gs_reg0 = ari(rbf='TPS', gtol=1e-8, n_samples=100, rel_ct='exclusive', sampling_method='random', random_state=57, print_debug=False)

In [ ]:
tps_ari_param_grid0 = {
    #'w0_arr': w0_patterns,
    'lambda_val': [0, 0.1, 0.5, 1, 2, 10]
    #'n_samples': [10, 20, 30],
    #'sampling_method': ['random', 'last']
}

In [ ]:
# grid searchの実行
gs0 = GridSearchCV(
    gs_reg0,
    param_grid=tps_ari_param_grid0,
    scoring='neg_mean_absolute_error',
    # cv=2
    cv=gs_kf
)
gs0.fit(train_x_arr, train_y_arr)

In [ ]:
# 最適なパラメータとスコアの確認
gs0.best_params_, gs0.best_score_, gs0.best_estimator_.params

In [ ]:
gs0.cv_results_

In [ ]:
best_tps_ari_params0 = gs0.best_estimator_.get_params()
best_tps_ari_params0

In [ ]:
gs_reg1 = ari(**best_tps_ari_params0)
gs_reg1.fit(train_x_arr, train_y_arr)
gs_reg1.get_params()

In [ ]:
# 重み初期値候補
w0_patterns = list(stats.temp_multivar_normal_dist_sampling(gs_reg1.params['n_samples'], 3, mean_val=1, var_val=4))
w0_patterns += [np.zeros(gs_reg1.params['n_samples'])]
print(w0_patterns[:5])

In [ ]:
tps_ari_param_grid1 = {
    'w0_arr': w0_patterns
}

In [ ]:
# 重み初期値についてgrid searchを実行する.
gs1 = GridSearchCV(
    gs_reg1,
    param_grid=tps_ari_param_grid1,
    scoring='neg_mean_absolute_error',
    # cv=2
    cv=gs_kf
)
gs1.fit(train_x_arr, train_y_arr)

In [ ]:
# 最適なパラメータとスコアの確認
gs1.best_params_, gs1.best_score_

In [ ]:
best_tps_ari_params1 = gs1.best_estimator_.get_params()
best_tps_ari_params1

In [ ]:
utils.get_elapsed_time(tps_gs_start)

### modeling

In [ ]:
tps_ari_params0 = best_tps_ari_params1

In [ ]:
# best paramsを設定してモデリング
tps_ari_reg = ari(**tps_ari_params0)
tps_ari_reg.fit(train_x_arr, train_y_arr)

In [ ]:
tps_ari_reg.get_params()

In [ ]:
tps_aripred_train_y_arr = tps_ari_reg.predict(train_x_arr)
tps_aripred_test_y_arr = tps_ari_reg.predict(test_x_arr)

# Validation

## train

In [ ]:
mae(train_y_arr, a2fpred_train_y_arr)

In [ ]:
mae(train_y_arr, gaussian_aripred_train_y_arr)

In [ ]:
mae(train_y_arr, tps_aripred_train_y_arr)

In [ ]:
pred_train_df = pd.DataFrame(np.vstack([
    train_y_arr,
    a2fpred_train_y_arr,
    gaussian_aripred_train_y_arr,
    tps_aripred_train_y_arr
]).T, columns=['train_y', 'a2fpred_train_y', 'gaussian_aripred_train_y', 'tps_aripred_train_y'])
pred_train_df

In [ ]:
pred_train_df.describe()

In [ ]:
(pred_train_df['gaussian_aripred_train_y']/pred_train_df['train_y']).describe()

In [ ]:
(pred_train_df['gaussian_aripred_train_y'] - pred_train_df['train_y']).describe()

In [ ]:
abs(pred_train_df['gaussian_aripred_train_y'] - pred_train_df['train_y']).describe()

In [ ]:
(pred_train_df['tps_aripred_train_y']/pred_train_df['train_y']).describe()

In [ ]:
(pred_train_df['tps_aripred_train_y'] - pred_train_df['train_y']).describe()

In [ ]:
abs(pred_train_df['tps_aripred_train_y'] - pred_train_df['train_y']).describe()

## test

In [ ]:
mae(test_y_arr, a2fpred_test_y_arr)

In [ ]:
mae(test_y_arr, gaussian_aripred_test_y_arr)

In [ ]:
mae(test_y_arr, tps_aripred_test_y_arr)

In [ ]:
pred_test_df = pd.DataFrame(np.vstack([
    test_y_arr,
    a2fpred_test_y_arr,
    gaussian_aripred_test_y_arr,
    tps_aripred_test_y_arr
]).T, columns=['test_y', 'a2fpred_test_y', 'gaussian_aripred_test_y', 'tps_aripred_test_y'])
pred_test_df

In [ ]:
pred_test_df.describe()

In [ ]:
(pred_test_df['gaussian_aripred_test_y']/pred_test_df['test_y']).describe()

In [ ]:
(pred_test_df['gaussian_aripred_test_y'] - pred_test_df['test_y']).describe()

In [ ]:
abs(pred_test_df['gaussian_aripred_test_y'] - pred_test_df['test_y']).describe()

In [ ]:
(pred_test_df['tps_aripred_test_y']/pred_test_df['test_y']).describe()

In [ ]:
(pred_test_df['tps_aripred_test_y'] - pred_test_df['test_y']).describe()

In [ ]:
abs(pred_test_df['tps_aripred_test_y'] - pred_test_df['test_y']).describe()

## matome

In [ ]:
pred_train_df.columns.tolist()

In [ ]:
matome_valid = pd.DataFrame([
    ['a2f', 'train', mae(pred_train_df['train_y'], pred_train_df['a2fpred_train_y'])],
    ['a2f', 'test', mae(pred_test_df['test_y'], pred_test_df['a2fpred_test_y'])],
    ['gaussian_ari', 'train', mae(pred_train_df['train_y'], pred_train_df['gaussian_aripred_train_y'])],
    ['gaussian_ari', 'test', mae(pred_test_df['test_y'], pred_test_df['gaussian_aripred_test_y'])],
    ['tps_ari', 'train', mae(pred_train_df['train_y'], pred_train_df['tps_aripred_train_y'])],
    ['tps_ari', 'test', mae(pred_test_df['test_y'], pred_test_df['tps_aripred_test_y'])]
], columns=['method', 'train_test', 'mae'])
matome_valid

# 2D sample フィッティング予実可視化

## 2d function

### train

In [ ]:
a2fpred_train_Y = a2fpred_train_y_arr.reshape([train_n_sample_grid, train_n_sample_grid])
a2fpred_train_Y.shape

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('2d approximation train data', size=15)

# 軸ラベルを設定
ax.set_xlabel('train_x1', size=16)
ax.set_ylabel('train_x2', size=16)
ax.set_zlabel('pred_train_y', size=16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, a2fpred_train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, a2fpred_train_Y, colors='black', offset=-1)

plt.show()

### test

In [ ]:
a2fpred_test_Y = a2fpred_test_y_arr.reshape([test_n_sample_grid, test_n_sample_grid])
a2fpred_test_Y.shape

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('2d approximation test data', size=15)

# 軸ラベルを設定
ax.set_xlabel('test_x1', size=16)
ax.set_ylabel('test_x2', size=16)
ax.set_zlabel('pred_test_y', size=16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, a2fpred_test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, a2fpred_test_Y, colors='black', offset=-1)

plt.show()

### 曲面の全容が見たい

In [ ]:
a2fpred_long_y_arr = a2f_reg.predict(long_x_arr)
a2fpred_long_Y = a2fpred_long_y_arr.reshape([long_n_sample_grid, long_n_sample_grid])

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('2d approximation long data', size=15)

# 軸ラベルを設定
ax.set_xlabel('long_x1', size=16)
ax.set_ylabel('long_x2', size=16)
ax.set_zlabel('pred_long_y', size=16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, a2fpred_long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, a2fpred_long_Y, colors='black', offset=-1)

plt.show()

### 並べてみる

In [ ]:
n_fig_cols = 3
n_fig_rows = 2
width = 12
height = 9
title_size = 25

# width, height
fig = plt.figure(
    figsize=(n_fig_cols*width, n_fig_rows*height),
    facecolor='white',
    dpi=200
)

# 余白を設定
plt.subplots_adjust(wspace=0.05, hspace=0.1, left=0.05, right=0.95, bottom=0.05, top=0.95)



# --------------------------------------------------------------------------------
# train
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 1, projection='3d')

ax.set_title('2D sample train', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('train_y', size = 16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, train_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# test
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 2, projection='3d')

ax.set_title('2D sample test', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('test_y', size = 16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, test_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# wide
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 3, projection='3d')

ax.set_title('2D sample wide', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('wide_y', size = 16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, long_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# train pred
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 4, projection='3d')

ax.set_title('2d approximation predict train', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('train_x1', size=16)
ax.set_ylabel('train_x2', size=16)
ax.set_zlabel('pred_train_y', size=16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, a2fpred_train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, a2fpred_train_Y, colors='black', offset=-1)


# --------------------------------------------------------------------------------
# test pred
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 5, projection='3d')

ax.set_title('2d approximation predict test', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size=16)
ax.set_ylabel('x2', size=16)
ax.set_zlabel('pred_test_y', size=16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, a2fpred_test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, a2fpred_test_Y, colors='black', offset=-1)


# --------------------------------------------------------------------------------
# wide pred
# --------------------------------------------------------------------------------
# longをwideとする

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 6, projection='3d')

ax.set_title('2d approximation predict widely', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size=16)
ax.set_ylabel('x2', size=16)
ax.set_zlabel('pred_wide_y', size=16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, a2fpred_long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, a2fpred_long_Y, colors='black', offset=-1)

plt.show()

## Gaussian RBF

### train

In [ ]:
gaussian_aripred_train_Y = gaussian_aripred_train_y_arr.reshape([train_n_sample_grid, train_n_sample_grid])
gaussian_aripred_train_Y.shape

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('Gaussian RBF approximation train data', size=15)

# 軸ラベルを設定
ax.set_xlabel('train_x1', size=16)
ax.set_ylabel('train_x2', size=16)
ax.set_zlabel('pred_train_y', size=16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, gaussian_aripred_train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, gaussian_aripred_train_Y, colors='black', offset=-1)

plt.show()

### test

In [ ]:
gaussian_aripred_test_Y = gaussian_aripred_test_y_arr.reshape([test_n_sample_grid, test_n_sample_grid])
gaussian_aripred_test_Y.shape

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('Gaussian RBF approximation test data', size=15)

# 軸ラベルを設定
ax.set_xlabel('test_x1', size=16)
ax.set_ylabel('test_x2', size=16)
ax.set_zlabel('pred_test_y', size=16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, gaussian_aripred_test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, gaussian_aripred_test_Y, colors='black', offset=-1)

plt.show()

### 曲面の全容が見たい

In [ ]:
gaussian_aripred_long_y_arr = gaussian_ari_reg.predict(long_x_arr)
gaussian_aripred_long_Y = gaussian_aripred_long_y_arr.reshape([long_n_sample_grid, long_n_sample_grid])

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('Gaussian RBF approximation long data', size=15)

# 軸ラベルを設定
ax.set_xlabel('long_x1', size=16)
ax.set_ylabel('long_x2', size=16)
ax.set_zlabel('gaussian_aripred_long_y', size=16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, gaussian_aripred_long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, gaussian_aripred_long_Y, colors='black', offset=-1)

plt.show()

### 並べてみる

In [ ]:
n_fig_cols = 3
n_fig_rows = 2
width = 12
height = 9
title_size = 25

# width, height
fig = plt.figure(
    figsize=(n_fig_cols*width, n_fig_rows*height),
    facecolor='white',
    dpi=200
)

# 余白を設定
plt.subplots_adjust(wspace=0.05, hspace=0.1, left=0.05, right=0.95, bottom=0.05, top=0.95)



# --------------------------------------------------------------------------------
# train
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 1, projection='3d')

ax.set_title('2D sample train', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('train_y', size = 16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, train_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# test
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 2, projection='3d')

ax.set_title('2D sample test', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('test_y', size = 16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, test_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# wide
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 3, projection='3d')

ax.set_title('2D sample wide', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('wide_y', size = 16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, long_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# train pred
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 4, projection='3d')

ax.set_title('Gaussian RBF approximation predict train', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('train_x1', size=16)
ax.set_ylabel('train_x2', size=16)
ax.set_zlabel('pred_train_y', size=16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, gaussian_aripred_train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, gaussian_aripred_train_Y, colors='black', offset=-1)


# --------------------------------------------------------------------------------
# test pred
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 5, projection='3d')

ax.set_title('Gaussian RBF approximation predict test', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size=16)
ax.set_ylabel('x2', size=16)
ax.set_zlabel('pred_test_y', size=16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, gaussian_aripred_test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, gaussian_aripred_test_Y, colors='black', offset=-1)


# --------------------------------------------------------------------------------
# wide pred
# --------------------------------------------------------------------------------
# longをwideとする

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 6, projection='3d')

ax.set_title('Gaussian RBF approximation predict widely', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size=16)
ax.set_ylabel('x2', size=16)
ax.set_zlabel('pred_wide_y', size=16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, gaussian_aripred_long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, gaussian_aripred_long_Y, colors='black', offset=-1)

plt.show()

## Thin Plate Spline RBF

### train

In [ ]:
tps_aripred_train_Y = tps_aripred_train_y_arr.reshape([train_n_sample_grid, train_n_sample_grid])
tps_aripred_train_Y.shape

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('Thin Plate Spline RBF approximation train data', size=15)

# 軸ラベルを設定
ax.set_xlabel('train_x1', size=16)
ax.set_ylabel('train_x2', size=16)
ax.set_zlabel('pred_train_y', size=16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, tps_aripred_train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, tps_aripred_train_Y, colors='black', offset=-1)

plt.show()

### test

In [ ]:
tps_aripred_test_Y = tps_aripred_test_y_arr.reshape([test_n_sample_grid, test_n_sample_grid])
tps_aripred_test_Y.shape

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('Thin Plate Spline RBF approximation test data', size=15)

# 軸ラベルを設定
ax.set_xlabel('test_x1', size=16)
ax.set_ylabel('test_x2', size=16)
ax.set_zlabel('pred_test_y', size=16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, tps_aripred_test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, tps_aripred_test_Y, colors='black', offset=-1)

plt.show()

### 曲面の全容が見たい

In [ ]:
tps_aripred_long_y_arr = tps_ari_reg.predict(long_x_arr)
tps_aripred_long_Y = tps_aripred_long_y_arr.reshape([long_n_sample_grid, long_n_sample_grid])

In [ ]:
# Figureと3DAxeS
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.set_title('Thin Plate Spline RBF approximation long data', size=15)

# 軸ラベルを設定
ax.set_xlabel('long_x1', size=16)
ax.set_ylabel('long_x2', size=16)
ax.set_zlabel('tps_aripred_long_y', size=16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, tps_aripred_long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, tps_aripred_long_Y, colors='black', offset=-1)

plt.show()

### 並べてみる

In [ ]:
n_fig_cols = 3
n_fig_rows = 2
width = 12
height = 9
title_size = 25

# width, height
fig = plt.figure(
    figsize=(n_fig_cols*width, n_fig_rows*height),
    facecolor='white',
    dpi=200
)

# 余白を設定
plt.subplots_adjust(wspace=0.05, hspace=0.1, left=0.05, right=0.95, bottom=0.05, top=0.95)



# --------------------------------------------------------------------------------
# train
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 1, projection='3d')

ax.set_title('2D sample train', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('train_y', size = 16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, train_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# test
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 2, projection='3d')

ax.set_title('2D sample test', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('test_y', size = 16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, test_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# wide
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 3, projection='3d')

ax.set_title('2D sample wide', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size = 16)
ax.set_ylabel('x2', size = 16)
ax.set_zlabel('wide_y', size = 16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, long_Y, colors = 'black', offset = -1)


# --------------------------------------------------------------------------------
# train pred
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 4, projection='3d')

ax.set_title('Thin Plate Spline RBF approximation predict train', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('train_x1', size=16)
ax.set_ylabel('train_x2', size=16)
ax.set_zlabel('pred_train_y', size=16)

# 曲面を描画
ax.plot_surface(train_X1, train_X2, tps_aripred_train_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(train_X1, train_X2, tps_aripred_train_Y, colors='black', offset=-1)


# --------------------------------------------------------------------------------
# test pred
# --------------------------------------------------------------------------------

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 5, projection='3d')

ax.set_title('Thin Plate Spline RBF approximation predict test', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size=16)
ax.set_ylabel('x2', size=16)
ax.set_zlabel('pred_test_y', size=16)

# 曲面を描画
ax.plot_surface(test_X1, test_X2, tps_aripred_test_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(test_X1, test_X2, tps_aripred_test_Y, colors='black', offset=-1)


# --------------------------------------------------------------------------------
# wide pred
# --------------------------------------------------------------------------------
# longをwideとする

# Figureと3DAxeS
ax = fig.add_subplot(n_fig_rows, n_fig_cols, 6, projection='3d')

ax.set_title('Thin Plate Spline RBF approximation predict widely', size=title_size)

# 軸ラベルを設定
ax.set_xlabel('x1', size=16)
ax.set_ylabel('x2', size=16)
ax.set_zlabel('pred_wide_y', size=16)

# 曲面を描画
ax.plot_surface(long_X1, long_X2, tps_aripred_long_Y, cmap='jet')

# 底面に等高線を描画
ax.contour(long_X1, long_X2, tps_aripred_long_Y, colors='black', offset=-1)

plt.show()

# Optimization

## Objective scipy.optimize

In [ ]:
def minus_reg(x_list, reg=None, n_divide=1):
    """
    小さくしないとエラーが起きることがある
    Massage: Positive directional derivative for linesearch
    Remark: https://stackoverflow.com/questions/11155721/positive-directional-derivative-for-linesearch
    """
    ret = reg.predict(np.array([x_list])) / n_divide
    return - ret

## 制約条件

Constraint全般  
https://qiita.com/imaizume/items/44896c8e1dd0bcbacdd5

LinearConstraint(線形制約条件について)  
https://qiita.com/shotoyoo/items/83297f959e1922465288

In [ ]:
x_bound = (-15, 15) # 0以上1以下
x_bounds_list = [x_bound, x_bound]

In [ ]:
# 線形制約条件
lc_A = np.array([np.ones(n_dim)])
lb = np.array([-100]) # 下限
ub = np.array([100]) # 上限

cons = LinearConstraint(lc_A, lb, ub) # 総和がsum_ratioになる

## 初期解

In [ ]:
init_x = np.zeros(n_dim)
init_x

## Optimization

### 2d approximation

In [ ]:
a2f_opt_result = minimize(fun=(lambda x: minus_reg(x, reg=a2f_reg, n_divide=10)), x0=init_x, bounds=x_bounds_list, constraints=cons, method='SLSQP')
a2f_opt_result

In [ ]:
a2f_opt_result.x

In [ ]:
a2f_pred_max = a2f_reg.predict([a2f_opt_result.x])
a2f_pred_max

#### 予測値のmax, 実際のmaxと比較する

In [ ]:
a2fpred_test_y_arr.max()

In [ ]:
test_y_arr.max()

### Gaussian RBF

In [ ]:
gaussian_ari_opt_result = minimize(fun=(lambda x: minus_reg(x, reg=gaussian_ari_reg, n_divide=10)), x0=init_x, bounds=x_bounds_list, constraints=cons, method='SLSQP')
gaussian_ari_opt_result

In [ ]:
gaussian_ari_opt_result.x

In [ ]:
gaussian_ari_pred_max = gaussian_ari_reg.predict([gaussian_ari_opt_result.x])
gaussian_ari_pred_max

In [ ]:
gaussian_aripred_test_y_arr.max()

### Thin Plate Spline RBF

In [ ]:
tps_ari_opt_result = minimize(fun=(lambda x: minus_reg(x, reg=tps_ari_reg, n_divide=10)), x0=init_x, bounds=x_bounds_list, constraints=cons, method='SLSQP')
tps_ari_opt_result

In [ ]:
tps_ari_opt_result.x

In [ ]:
tps_ari_pred_max = tps_ari_reg.predict([tps_ari_opt_result.x])
tps_ari_pred_max

In [ ]:
tps_aripred_test_y_arr.max()

# Check

## get elapsed time

In [ ]:
utils.get_elapsed_time(start_time)